In [28]:
import pandas as pd
import os
from tqdm import tqdm
from multiprocessing import Pool 
import numpy as np

In [47]:
OUT_STEM = "/shared/3/projects/benlitterer/podcastData/NER/tempFloydIntroductions/floydIntroductions"

In [2]:
df = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthEnSHORT.csv") 

/tmp/ipykernel_3389318/586084763.py:1: DtypeWarning: Columns (6,7,8,12,20,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthEnSHORT.csv")


In [ ]:
df["potentialOutPathFull"] = "/shared/3/projects/benlitterer/podcastData/prosodyMerged/floydMonth" + df["potentialOutPath"] 
df["exists"] = df["potentialOutPathFull"].apply(lambda x: os.path.exists(x)) 
df = df[df["exists"] == True]

In [29]:
dfList = np.array_split(df, 12)

In [43]:
def outputDataFile(i): 
    currDf = dfList[i]
    OUT_PATH = f"{OUT_STEM}{i}.tsv"
    with open(OUT_PATH, "w") as outFile: 
        for path in tqdm(currDf["potentialOutPathFull"]): 
            with open(path) as currFile: 
                fullText =  "".join([row.split(",")[4] for row in currFile.readlines()])
                firstBit = " ".join(fullText.split()[:500]).replace("\t", "").replace("\n", "") 
                outFile.write(f"{path}\t{firstBit}\n")

In [48]:
pool = Pool(processes=12)
pool.map(outputDataFile, list(range(12)))

  1%|█▎                                                                                                     | 570/44935 [00:49<1:04:44, 11.42it/s]
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda/lib/python3.11/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3389318/409557742.py", line 5, in outputDataFile
    for path in tqdm(currDf["potentialOutPathFull"]):
  File "/opt/anaconda/lib/python3.11/site-packages/tqdm/std.py", line 1192, in __iter__
    self.update(n - last_print_n)
  File

KeyboardInterrupt: 

In [53]:
catDf = pd.read_csv("/shared/3/projects/benlitterer/podcastData/NER/tempFloydIntroductions/floydIntroductions.tsv", sep="\t", names=["outPath", "content"]) 